# Lab-11-2 RNN hihello and charseq

In [1]:
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

## Data

In [2]:
sample = "can anybody find me somebody to love?"

In [3]:
char_set = list(set(sample))
char_dic = {c: i for i, c in enumerate(char_set)}
char_dic

{'l': 0,
 'm': 1,
 '?': 2,
 's': 3,
 'n': 4,
 'a': 5,
 'v': 6,
 'e': 7,
 'c': 8,
 'i': 9,
 'y': 10,
 'b': 11,
 'o': 12,
 'd': 13,
 't': 14,
 ' ': 15,
 'f': 16}

In [4]:
dic_size = len(char_dic)

In [5]:
sample_idx = [char_dic[c] for c in sample]
sample_idx

[8,
 5,
 4,
 15,
 5,
 4,
 10,
 11,
 12,
 13,
 10,
 15,
 16,
 9,
 4,
 13,
 15,
 1,
 7,
 15,
 3,
 12,
 1,
 7,
 11,
 12,
 13,
 10,
 15,
 14,
 12,
 15,
 0,
 12,
 6,
 7,
 2]

In [6]:
x_data = [sample_idx[:-1]]
x_data

[[8,
  5,
  4,
  15,
  5,
  4,
  10,
  11,
  12,
  13,
  10,
  15,
  16,
  9,
  4,
  13,
  15,
  1,
  7,
  15,
  3,
  12,
  1,
  7,
  11,
  12,
  13,
  10,
  15,
  14,
  12,
  15,
  0,
  12,
  6,
  7]]

In [7]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
x_one_hot

[array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.

In [8]:
y_data = [sample_idx[1:]]
y_data

[[5,
  4,
  15,
  5,
  4,
  10,
  11,
  12,
  13,
  10,
  15,
  16,
  9,
  4,
  13,
  15,
  1,
  7,
  15,
  3,
  12,
  1,
  7,
  11,
  12,
  13,
  10,
  15,
  14,
  12,
  15,
  0,
  12,
  6,
  7,
  2]]

In [9]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print(X)
print(Y)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

## Hyperparameters

In [10]:
input_size = dic_size
hidden_size = dic_size
learning_rate = 1e-1

## Model

In [11]:
rnn = nn.RNN(input_size, hidden_size, batch_first=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

## Train and Predict

In [12]:
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    
    if epoch % 10 == 0:
        print('Epoch:', epoch, 'Loss:', loss.item())
        print(result_str)

Epoch: 0 Loss: 2.815047264099121
i e yedeyeviv eyieyeymeyedevieeeeeey
Epoch: 10 Loss: 1.7790802717208862
an  ny fdn fie   fb omebfdn fo love?
Epoch: 20 Loss: 1.4867793321609497
an any ody fiadyme some ody fo love?
Epoch: 30 Loss: 1.3800584077835083
an any ody tind mebsomebody to love?
Epoch: 40 Loss: 1.3302500247955322
an any ody tindymebsomebody to love?
Epoch: 50 Loss: 1.3018155097961426
an any ody find mebsomebody fo love?
Epoch: 60 Loss: 1.2866445779800415
an any ody tindymebsomebody to love?
Epoch: 70 Loss: 1.2764923572540283
an any ody tindymebsomebody to love?
Epoch: 80 Loss: 1.2691245079040527
an any ody tind me somebody to love?
Epoch: 90 Loss: 1.2629609107971191
an any ody tind me somebody to love?
